In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import warnings
from urllib.request import urlopen
import json

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
warnings.simplefilter("ignore")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Reading data into python -- cases data 
cases = pd.read_csv("/kaggle/input/covid19-data-from-john-hopkins-university/CONVENIENT_us_confirmed_cases.csv"
                    ,index_col= False
                    ,header = None
                   ).transpose()

#adding column names to the cases data 
cases.columns = cases.iloc[0]
cases = cases.iloc[1:,0:]
cases.head()

In [ ]:
#deaths data 
deaths = pd.read_csv("/kaggle/input/covid19-data-from-john-hopkins-university/CONVENIENT_us_deaths.csv"
                    ,index_col= False
                    ,header = None
                   ).transpose()

#chaning column names to the first row values 
deaths.columns = deaths.iloc[0]
deaths = deaths.iloc[1:,0:]
deaths.head()

In [ ]:
#Reading metadata into python 
cases_md = pd.read_csv("/kaggle/input/covid19-data-from-john-hopkins-university/CONVENIENT_us_metadata.csv")

#fips data to get the FIPS id 
fips = pd.read_csv("/kaggle/input/covid19-data-from-john-hopkins-university/RAW_us_confirmed_cases.csv").iloc[:,0:7]

#filling null with 0s 
fips.FIPS = fips.FIPS.fillna(0) #replacing NA values with 0
fips.head()

In [ ]:
# appending metadata to original data 
merged_data = pd.merge(cases_md, cases, on = ('Province_State','Admin2'), how = 'left')

#appending FIPS data to the metadata + original data - FIPS id needs to be used in the mapping 
merged_data = pd.merge(fips,merged_data, on = ('Province_State','Admin2'),how = 'left')

merged_data.head()

In [ ]:
#creating a dataset with all the value that we want 
fd = pd.concat(
                 [ merged_data[['Province_State','Admin2','Population','FIPS','Lat','Long']]
                  ,merged_data.iloc[:,10:].astype(float).sum(axis = 1)] #this code gives the sum     
                  ,axis = 1)

#chaning the column name from 0 to Total cases 
fd =  fd.rename(columns = {0:'Total_cases'})

# changing FIPS column from float to string and appending leading zeros to be in compliance with the FIPS format 
fd.FIPS = fd.FIPS.astype(int).astype(str).str.zfill(5) 


fd.head()


In [ ]:
#choropelth Maps - county level data

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


fig = px.choropleth_mapbox(fd,geojson=counties , locations = 'FIPS', color='Total_cases',
                           color_continuous_scale= "earth", #"Viridis",
                           range_color=(0,4000),
                           mapbox_style= "carto-darkmatter", #"carto-positron","open-street-map",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           hover_data=["Province_State", "Admin2"],
                           template = 'plotly_dark',
                           #title = 'COVID - 19 Cases Across USA - County View',
                           labels={'Total_cases': 'COVID Cases'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


fig.show()

In [ ]:
nd = merged_data.iloc[:,10:].T #this has confirmed cases data 

dd = deaths.iloc[:,3:].T #this has deaths data 

cases_deaths = pd.merge(
                        pd.DataFrame(nd.astype(float).sum(axis = 1),columns = ["confirmed_cases"])
                       ,pd.DataFrame(dd.astype(float).sum(axis = 1),columns = ["Deaths"])
                       ,left_index=True
                       ,right_index=True
                       )
                      

cases_deaths.head()

In [ ]:
# Confirmed Cases Data -- line graphs 

fig = px.line( 
                cases_deaths, 
                x= cases_deaths.index.values
               ,y= 'confirmed_cases'
               ,line_shape = 'spline'
               ,title= "COVID 19 confirmed cases across USA"
               ,template = 'plotly_dark'
            )

fig.update_layout(
    title={
        'text': "COVID 19 confirmed cases across USA",
        #'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title = '',
    yaxis_title="Count"
    
)

fig.show()

In [ ]:
#bar graphs 

fig = px.bar(x= cases_deaths.index.values, 
             y = cases_deaths.Deaths,
             template = 'plotly_dark')

fig.update_layout(
    title={
        'text': "Deaths Related to COVID 19 Across USA",
        #'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title = '',
    yaxis_title="Count"
    
)     

fig.show()